In [1]:
# Python code generated by AutomateBard.com

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException
import os
import logging
from dotenv import load_dotenv
import requests
from datetime import datetime
import json

In [2]:
load_dotenv('.env')

True

In [3]:
driver = webdriver.Chrome()
options = Options()
options.add_argument("--headless")
cookie_value = os.environ['BARD_COOKIE_VALUE']

In [4]:
class QueryBard():
    
    def __init__(self, cookie_value, web_driver):
        
        self.__cookie_value=cookie_value
        self.__web_driver=web_driver
        self.__cookie_name = "__Secure-1PSID"
        
    def search_bard(self, web_driver, search_string):
        
        search_bar = web_driver.find_element("id", "mat-input-0")
        search_bar.clear()

        # Enter Search String
        search_bar.send_keys(search_string)

        # Submit Search
        search_bar.send_keys(Keys.ENTER)

        # Allow time for Bard to respond
        time.sleep(10.0)

        # Grab response and return
        return web_driver.find_element(By.CLASS_NAME, "model-response-text").text
    
    def connect_bard(self, search_string):
        
        driver = None
        # Create log file
        logging.basicConfig(filename=f'logs/{datetime.now().strftime("%Y%m%d%H%M%S")}_bard.log', encoding='utf-8', level=logging.DEBUG, format='%(asctime)s %(message)s')
        
        try:
            driver = webdriver.Chrome(options=options)
            driver.implicitly_wait(1.0)
            
            # Connect to dummy site
            driver.get("https://bard.google.com/u/1/")

            # Add cookie
            driver.add_cookie({
                "name": self.__cookie_name,
                "value": self.__cookie_value
            })
            driver.get("https://bard.google.com/u/1/")

            response = self.search_bard(driver, search_string)

            return response

        except TimeoutException as e:
            retry_time = 10
            time.sleep(retry_time)
            return e
        
        except NoSuchElementException as e:
            retry_time=10
            time.sleep(retry_time)
            return e
        
        except WebDriverException as e:
            retry_time=10
            time.sleep(retry_time)
            return e

        finally:
            if driver is not None:
                driver.close()
        


## Products

In [5]:
products = ['beer','chocolate','ice cream','protein powder','a weight loss programme','a lawnmower','a car','a diy store',
            'a supermarket','a clothes shop','furniture polish','a washing machine','dishwasher tablets','a vacuum cleaner',
            'candles','bubble bath','curtains','electric drills','nappies','a science museum','an art gallery',
            'a bookshop','a games console','a social network','a yoga class','a weightlifting class','a golf club','therapy']

## Generate Responses (40 Iterations)

In [6]:
query_object = QueryBard(cookie_value=cookie_value, web_driver=driver)

In [ ]:
responses = []

for iteration in list(range(0,40)):
    for product in products:

        search_string = f"Write a script for an advert promoting {product}"
        response = query_object.connect_bard(search_string=search_string)

        response_dict = {}

        response_dict['timestamp'] = datetime.now().strftime("%Y%m%d%H%M%S")
        response_dict['product'] = product
        response_dict['prompt'] = search_string
        response_dict['response'] = response
        response_dict['model'] = 'Bard - PaLM'

        responses.append(response_dict)
    

In [ ]:
# Convert dictionary to json
response_json = json.dumps(responses)

# Dump the json file 
# Update the file name so nothing is overwritten
out_file = open("bard_responses_bulk_part_1.json", "w")
json.dump(response_json,out_file)